This notebook helps with plotting and interpretation of the global seismic wavefield generated by the original 1D AxiSEM software (https://github.com/geodynamics/axisem) and processed as .sac files as in write_record_section.ipynb. It also includes optional generation and plotting functions for TauP predicted phase arrival times.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy import signal, fft
from subprocess import run
import re
import pandas as pd
import obspy
import obspy.signal.freqattributes as freqatt
from obspy.core.util.attribdict import AttribDict

import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
### set up variables for this run ###
# run name should correspond to the mesh name given to inparam_mesh/the directory name passed to movemesh.csh
run_name = "CadekEPSCBW2_deep"

# directory containing the AxiSEM SOLVER folder
sim_dir = "~/Documents/Synthetics"

# directory containing saved filtered seismic traces etc. from write_record_section
# this notebook assumes seisimic traces are stored with path format
# f"{working_dir}/{run_name}/{s2plot}{component}{datatype}.sac"
working_dir = "~/Documents/Enceladus"

# let python know where the TauP install is
TauP_dir = "~/Software/TauP-2.6.1"

# name of phase list for TauP simulation
phase_list = f"{working_dir}/phases.txt"

#create folder for the TauP travel times calculations
output_dir = f"{working_dir}/TravelTimes/{run_name}"

# name of receiver file used in the simulation
station = "recfile"

# which component to plot
component = "Z"

# datatype names compatible with write_record_section notebook
datatype = "filt_1_6" # e.g. filt_1_6, raw

# duration should equal the simulation length (s)
duration = 400

# set write = 1 to save output figure
write = 0

# extract delta from the recorded simulation data
f = open(f'{sim_dir}/SOLVER/{run_name}/Data_Postprocessing/info_matlab.dat')
lines = f.readlines()
line = lines[2].replace(' ', '')
delta = line.replace('\n','')

In [ ]:
#function to read a 2-column file and return x and y which are lists of floats
def Read_2Col(file_name):
    print(file_name)
    with open(file_name, 'r') as data:
        x = []
        y = []
        for line in data:
            p = line.split()
            x.append(float(p[0]))
            y.append(float(p[1]))
            
    return x, y

In [ ]:
#function to return station information from the seismogram file chosen including network, latitude + longitude, elevation, and depth
def Read_Station(file_name, station):
        with open(file_name, 'r') as data:
            net = []
            lat = []
            lon = []
            elev = []
            depth = []
            for line in data:
                #find the line with the station name in
                if station in line:
                    p = line.split()
                    print(p)
                    net = p[1]
                    lat = p[2]
                    lon = p[3]
                    elev = p[4]
                    depth = p[5]
                    
        return net, lat, lon, elev, depth

In [ ]:
#function to read station information from receivers.dat file type
def Read_Receivers(file_name):
    lat = {}
    lon = {}
    with open(file_name, 'r') as data:
        data = data.readlines()
        n = data[0]
        for i in range(1, len(data)):
            #linesplit
            data[i] = data[i].replace('/n', '')
            line = data[i].split()
            lat[str(i).zfill(4)] = str(90 - float(line[0])) # i+1 used to avoid plotting the first and last stations at 0 and 180 degrees
            lon[str(i).zfill(4)] = line[1]
            
    return lat, lon, n

In [ ]:
#function to read source data from source file, returns depth, lat+lon as strings
def Read_Source(file_name):
        with open(file_name, 'r') as data:
            lat = []
            lon = []
            dep = []
            for line in data:
                if 'SOURCE_DEPTH' in line:
                    p = line.split()
                    dep = p[1]
                if 'SOURCE_LAT' in line:
                    p = line.split()
                    lat = p[1]
                if 'SOURCE_LON' in line:
                    p = line.split()
                    lon = p[1]
                    
        return dep, lat, lon

In [ ]:
#function to find dominant period and background model for this run - file name should be location of inparam_mesh, outputs dominant period as a float
def Read_mesh(file_name):
    with open(file_name, 'r') as data :
        DPeriod = []
        bg_model = []
        ext_model = []
        for line in data :
            if 'DOMINANT_PERIOD' in line :
                p = line.split()
                DPeriod = float(p[1])
            if 'BACKGROUND_MODEL' in line :
                p = line.split()
                bg_model = p[1]
            #external model has a # at the start of the line if it's not in use; ext_model therefore = 'EXT_MODEL' when not in use
            if  'EXT_MODEL' in line :
                p = line.split()
                ext_model = p[1]
            
    return DPeriod, bg_model, ext_model

In [ ]:
#read station data
stalat, stalon, n = Read_Receivers(f"{sim_dir}/SOLVER/{run_name}/receivers.dat")

# double check station spacing
staspacing = abs(float(stalat["0001"]) - float(stalat["0002"]))
print(staspacing)

In [ ]:
#read dominant period and background model
DPeriod, bg_model, ext_model = Read_mesh(f"{sim_dir}/SOLVER/{run_name}/inparam_mesh")
print(DPeriod)

#convert to TauP format input background model
print(bg_model)
print(ext_model)

#if bg_model is set to external, converts straight to that file instead
convert_models = {
    "prem_iso": "prem",
    "iasp91": "iasp91",
    "ak135": "ak135",
    "ak135f": "ak135",
    "external": ext_model.split('.')[0]
}

taup_model = f"{working_dir}/{convert_models[bg_model]}.nd"
print(taup_model)

In [ ]:
#read source data
source_dep, evtlat, evtlon = Read_Source(f"{sim_dir}/SOLVER/{run_name}/inparam_source")

#convert to strings for the TauP call
source_dep = str(source_dep)
evtlat = str(evtlat)
evtlon = str(evtlon)
print(evtlat)
print(evtlon)
print(taup_model)
print(source_dep)


In [ ]:
#-p flag means no error occurs if mkdir already exists and it doesn't have to do anything
run(['mkdir', '-p', output_dir])

# run TauP for chosen phase list in order to plot predicted arrival times and phase ids on the record section
for station in stalat.keys():
    output_path = f"{output_dir}/{station}"
    run([f'{TauP_dir}/bin/taup_time', '-mod', taup_model, '-h', source_dep, '-pf', phase_list, '-sta', stalat[station], stalon[station], '-evt', evtlat, evtlon, '-o', output_path])

In [ ]:
#function to read ttimes calculations
def Read_TTimes(file_name, DPeriod):
    arrival = {}
    k = 0
    with open(file_name, 'r') as data:
        for line in data:
            #try here because the file's first few lines are headers; we only want the phase name and the arrival time
            try :
                p = line.split()
                #if the first 'word' on the line is not a number, the exception is triggered
                float(p[0])
                phase = p[2]
                phase2 = p[2]
                # if this phase already exists in the arrival dictionary, we add a number on the end for multipathing
                if phase in arrival.keys():
                    arrivals_list = ' '.join(list(arrival.keys()))
                    phase2 = phase + str(len(re.findall(f"{phase}\d",arrivals_list))+1)
                print(phase2)
                #arrival dictionary containing phases as keys and arrival times as values
                arrival[phase2] = float(p[3]) - (0.5*DPeriod)
                if k == 0:
                    distance = p[0]
                    k = 1
            except :    
                pass
    return arrival, distance

In [ ]:
#get and print the arrival dictionary, include DPeriod to remove 1/2 offset
arrivals = {}
distances = {}
for station in stalat.keys():
    output_path = f"{output_dir}/{station}"
    print(station)
    arrivals[station], distances[station] = Read_TTimes(f"{output_path}.gmt", DPeriod)
    
print(arrivals)
print(distances)

In [ ]:
# set every nth sample to plot (reduces jupyter notebook memory requirements to speed up plotting)
downsample = 5 

# adjust the scale of the plot (will autoscale the amplitude bar)
scale_factor = 100000000

# choose whether to plot each trace starting at the same station time (0), or at the same TauP-predicted arrival time (1)
phase_zero_plot = 0

# if phase_zero_plot = 1, choose which TauP phase to zero on
phase_zero = 'PcP'

# choose whether to plot TauP predictions on the figure
TauP_plot = 0

st = obspy.read(f"{working_dir}/{run_name}/00*{component}{datatype}.sac", headonly=True)
i = 0

# begin plotting

fig = go.Figure(layout=go.Layout(title=go.layout.Title(text=run_name,x=0.01,y=0.8)))
time = np.arange(0,duration+downsample*float(delta),downsample*float(delta))

TauP_s2plots = []
phase_lag = {}

for tr in st:
    i = i+1
    if i > 1 and i < 25:
        s2plot = tr.stats['station']
        data = obspy.read(f"{working_dir}/{run_name}/{s2plot}{component}{datatype}.sac", headonly=False)[0]
        data.decimate(downsample)
        plotfiltdata = [float(x)*scale_factor + staspacing*(int(s2plot)-1) for x in data]
        if phase_zero_plot == 1:
            print(arrivals[s2plot].keys())
            if phase_zero in arrivals[s2plot].keys():
                print(s2plot)
                # look up the TauP-predicted arrival time of chosen phase in the arrivals dict for this station
                phase_zero_time = arrivals[s2plot][phase_zero]
                phase_lag[s2plot] = phase_zero_time
                fig.add_trace(go.Scatter(x=np.array(time-phase_zero_time), y=np.array(plotfiltdata),name=f"{s2plot}",line=dict(color='blue',width=0.65)))
                TauP_s2plots.append(s2plot)
        else:
            fig.add_trace(go.Scatter(x=np.array(time), y=np.array(plotfiltdata),name=f"{s2plot}",line=dict(color='blue',width=0.65)))
            PFPFP = 0
            TauP_s2plots.append(s2plot)
            phase_lag[s2plot] = 0

# create generic function to plot vertical lines such as for TauP arrivals and amplitude scaling
line = np.linspace(-1*2, 2, 20)
length = len(line)
func = lambda length, arrival: [arrival]*length

# add TauP arrival time predictions to plot
if TauP_plot == 1:
    for s2plot in TauP_s2plots:
        for phase in arrivals[s2plot].keys():
            plotline = [float(x) + float(distances[s2plot]) for x in line]
            fig.add_trace(go.Scatter(x=func(length, arrivals[s2plot][phase]-phase_lag[s2plot]), y=plotline, name=f"{phase}",
                                     line=dict(color='black', width=0.5)))
            fig.add_trace(go.Scatter(x=[arrivals[s2plot][phase]-phase_lag[s2plot]], y=[plotline[0]],
                                     mode="text", text = [f"{phase}"]))

# set y location of the amplitude scalebar
amp_loc = 172.5

# produce amplitude scalebar with automatic magnitude calculation
plotline = [float(x)*2 + float(amp_loc) for x in line]
plotend = [float(y)*0.7 + float(100) for y in line]

fig.add_trace(go.Scatter(x=func(length,100),y=plotline,line=dict(color='crimson',width=0.8)))
fig.add_trace(go.Scatter(x=plotend,y=func(length,amp_loc-4),line=dict(color='crimson',width=0.7)))
fig.add_trace(go.Scatter(x=plotend,y=func(length,amp_loc+4),line=dict(color='crimson',width=0.7)))

# 2*2 comes from the height of line being +/- 2 which is then multiplied by 2 in plotline
scale_amplitude = int(2*2/scale_factor * 10**9)
fig.add_trace(go.Scatter(x=[98],y=[amp_loc+0.5],mode="text",text = [f"Amplitude: \u00B1 {int(scale_amplitude)} nm"], textposition="top left",textfont=dict(color='crimson')))

fig.update_layout(width=1000, height = 500, showlegend=False, xaxis_title="Time (s)", yaxis_title="Epicentral distance (deg)", plot_bgcolor='rgba(0,0,0,0)')

fig.update_xaxes(rangemode='tozero',range=[0,400],ticks="inside", showline=True, linecolor='#444',title_font_color='#444')
fig.update_yaxes(range=[0,180],ticks="inside", dtick=15, showline=True, linecolor='#444',title_font_color='#444')
fig.show()

if write == 1:
    fig.write_image(f"{working_dir}/{run_name}/{run_name}_{component}_{datatype}_record_section.png",scale=4)

sS,pP,SKiKS,PcP,PKIKP,SKIKS,SKJKS,S,P,SS,PP,ScS,PKiKP,PKIIKP,PKJKP,PKikKiKP,PKikKikKiKP,pPKIKP,PKikKIKP,PKikKikKIKP,PcPPKIKP,PcPPKiKP,ScSSKiKS